In [1]:
from linkedin import Head_Hunter_9000

In [2]:
import time
import random

In [3]:
hh_9000 = Head_Hunter_9000()
hh_9000.login()

In [4]:
time.sleep(random.uniform(1,2))
hh_9000.scroll_through_sidebar()
job_listings = hh_9000.driver.find_elements_by_xpath("//div[contains(@class, 'job-card-container') and contains(@class, 'job-card-list')]")

In [14]:
from selenium.common.exceptions import NoSuchElementException

In [17]:
def loop_through_questions(hh_9000, job_id):
    job_info_container = hh_9000.driver.find_element_by_xpath("//div[@class='job-view-layout jobs-details']")
    job_short = job_info_container.find_element_by_xpath(".//div[@class='jobs-unified-top-card__content--two-pane']")
    job_short.find_element_by_xpath(".//button[contains(@class, 'jobs-apply-button')]").click() # click easy apply
    # Loop through questions  //button[@data-easy-apply-next-button]
    more_questions = True
    while(more_questions):
        dd_and_fr_question_forms = hh_9000.driver.find_elements_by_xpath("//div[@class='jobs-easy-apply-content']//div[@class='jobs-easy-apply-form-section__grouping']/div[contains(@class, 'jobs-easy-apply-form-element')]")
        rb_question_forms = hh_9000.driver.find_elements_by_xpath("//div[@class='jobs-easy-apply-content']//div[@class='jobs-easy-apply-form-section__grouping']/fieldset")

        for q_form in dd_and_fr_question_forms: # dropdown: fb-dropdown    ---   free-response: fb-single-line-text
            question_text = q_form.find_element_by_xpath("./label/span[not(@class='visually-hidden')]").text
            question_type = q_form.find_element_by_xpath("./div").get_attribute("class")

            # //div[@class='jobs-easy-apply-content']//div[@class='jobs-easy-apply-form-section__grouping']/div[contains(@class, 'jobs-easy-apply-form-element')]/div/select/option
            if question_type == "fb-dropdown":
                
                options = q_form.find_elements_by_xpath("./div/select/option")
                option_vals = [option.get_attribute("value") for option in options]

                option_sets = hh_9000.litehelper.get_from_table("Questions q INNER JOIN DropDownQuestions d on q.QuestionID = d.DropDownQuestionID", ['OptionSetID'], {"Question": question_text})
                if option_sets: # question has already been added
                    for option_set in option_sets:
                        db_options = hh_9000.litehelper.get_from_table("Options", ['OptionText', 'Value'], {"OptionSetID": option_set})
                        db_option_texts = set()
                        if all([option_text in db_option_texts for option_text in option_texts]) and all([option_val in db_option_vals for option_val in option_vals]):
                            q_id = hh_9000.litehelper.get_from_table("DropDownQuestions", ['DropDownQuestionID'], {"OptionSet": option_set})
                            hh_9000.litehelper.insert_job_question(**{'JobID': job_id, 'QuestionID': q_id})
                            break
                    
                        
                    

                # if question is in db
                    # store all option_sets in a list of sets
                    # then for each set check if all options in the UI are in that set
                    # if ALL options are in that set, then this is the correct set and we just need to store reference to the OptionSetID
                    # else we need to add the same question to the db - this time with the new option set
                # else we need to add it to the db along with all the options
                
                

            print(question_text, question_type)

        for q_form in rb_question_forms:
            question_text = q_form.find_element_by_xpath("./legend/span[not(@class='visually-hidden')]").text
            options = q_form.find_element_by_xpath("./div[@class='fb-radio-buttons']")
            print(question_text)

        next_q_button = find_element_by_xpath_safe(hh_9000.driver, "//button[@data-easy-apply-next-button]")
        if next_q_button:
            next_q_button.click()
        else:
            more_questions = False

def find_element_by_xpath_safe(element, xpath_from_element):
    try:
        return element.find_element_by_xpath(xpath_from_element)
    except NoSuchElementException:
        return None

In [18]:
for i in range(len(job_listings)):
    just_added = False
    time.sleep(random.uniform(1,2))
    link = job_listings[i].find_element_by_xpath(".//a[contains(@class, 'job-card-container__link') and contains(@class, 'job-card-list__title')]")
    link.click()
    ext_job_id = job_listings[i].get_attribute("data-job-id")
    if not hh_9000.litehelper.is_in_table('Jobs', **{'JobBoardID':'linkedin', 'ExtJobID': ext_job_id}):
        job_info = hh_9000.build_job_info(ext_job_id)
        hh_9000.litehelper.insert_job(**job_info)
        just_added = True

    if just_added or hh_9000.litehelper.is_in_table('Jobs', **{'JobBoardID':'linkedin', 'ExtJobID': ext_job_id, 'AppSubmitted': 0}):
        loop_through_questions(hh_9000)
        break

            # If all Questions for Job already have Answers

            # Else if there are not yet any Questions

Email address fb-dropdown
Phone country code fb-dropdown
Mobile phone number fb-single-line-text
How many years of work experience do you have using Tableau? fb-single-line-text
How many years of work experience do you have using Python (Programming Language)? fb-single-line-text
Have you completed the following level of education: Bachelor's Degree?
Are you legally authorized to work in the United States?


### Old

In [5]:
job_listings = hh_9000.driver.find_elements_by_xpath("//div[contains(@class, 'job-card-container') and contains(@class, 'job-card-list')]")

In [8]:
job_info_container = hh_9000.driver.find_element_by_xpath("//div[@class='job-view-layout jobs-details']")
job_short = job_info_container.find_element_by_xpath(".//div[@class='jobs-unified-top-card__content--two-pane']")
easy_apply = job_short.find_element_by_xpath(".//button[contains(@class, 'jobs-apply-button')]").click()

In [25]:
dd_and_fr_question_forms = hh_9000.driver.find_elements_by_xpath("//div[@class='jobs-easy-apply-content']//div[@class='jobs-easy-apply-form-section__grouping']/div[contains(@class, 'jobs-easy-apply-form-element')]")
rb_question_forms = hh_9000.driver.find_elements_by_xpath("//div[@class='jobs-easy-apply-content']//div[@class='jobs-easy-apply-form-section__grouping']/fieldset")

In [26]:
for q_form in dd_and_fr_question_forms:
    question_text = q_form.find_element_by_xpath("./label/span[not(@class='visually-hidden')]").text
    question_type = q_form.find_element_by_xpath("./div").get_attribute("class")
    print(question_text, question_type)

In [27]:
for q_form in rb_question_forms:
    question_text = q_form.find_element_by_xpath("./legend/span[not(@class='visually-hidden')]")
    options = q_form.find_element_by_xpath("./div[@class='fb-radio-buttons']")
    print(question_text)

Are you comfortable commuting to this job's location?
